In [1]:
import streamlit as st
import yfinance as yf
from prophet import Prophet
from prophet.plot import plot_plotly
import pandas as pd

st.set_page_config(page_title="📈 Stock Price Forecast App")
st.title("📈 Stock Price Forecast App")

# Sidebar inputs
st.sidebar.header("Settings")
ticker = st.sidebar.text_input("Enter stock ticker", "AAPL").upper()

forecast_options = {
    "1 Week": 7,
    "1 Month": 30,
    "3 Months": 90,
    "6 Months": 180,
    "1 Year": 365
}
forecast_choice = st.sidebar.selectbox("Forecast period", list(forecast_options.keys()))
period = forecast_options[forecast_choice]

@st.cache(ttl=3600)
def load_data(ticker_symbol):
    try:
        data = yf.download(ticker_symbol, auto_adjust=True)
        data.reset_index(inplace=True)
        return data
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return None

df = load_data(ticker)

if df is not None and not df.empty:
    # Filter to last 3 years max to avoid huge data and memory issues
    max_history_days = 365 * 3
    df = df.tail(max_history_days)

    # Find Close price column
    close_col = None
    if isinstance(df.columns, pd.MultiIndex):
        # Flatten MultiIndex columns
        df.columns = [' '.join(col).strip() for col in df.columns.values]
    for col in df.columns:
        if 'Close' in col:
            close_col = col
            break

    if close_col is None:
        st.error("Close price column not found in data.")
    else:
        df_train = df[['Date', close_col]].rename(columns={'Date': 'ds', close_col: 'y'})
        df_train['y'] = pd.to_numeric(df_train['y'], errors='coerce')
        df_train = df_train.dropna(subset=['y'])

        st.subheader(f"Data used for forecasting ({len(df_train)} rows):")
        st.write(df_train.tail())

        try:
            m = Prophet(daily_seasonality=False)
            m.fit(df_train)

            future = m.make_future_dataframe(periods=period)
            forecast = m.predict(future)

            st.subheader("Forecast")
            fig1 = plot_plotly(m, forecast)
            st.plotly_chart(fig1, use_container_width=True)

            st.subheader("Forecast Components")
            fig2 = m.plot_components(forecast)
            st.pyplot(fig2)

            st.subheader("Forecast Data")
            st.write(forecast.tail())

        except Exception as e:
            st.error(f"Error during forecasting or plotting: {e}")
else:
    st.warning("No data available. Please check the stock ticker symbol.")


2025-05-26 22:15:12.568 
  command:

    streamlit run C:\Users\venum\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-26 22:15:12.587 Session state does not function when running a script without `streamlit run`


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `load_data()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function load_data at 0x000001A4EFCE0EE0>
```

Please see the `hash_funcs` [documentation]
(https://docs.streamlit.io/library/advanced-features/caching#the-has